# Import librerie

In [ ]:
import requests
import sys

import numpy as np

from pyspark.sql.functions import when, count, isnan, isnull, expr, col, current_date, date_format, year, month, lag, sum, round, max, min, udf, datediff, explode, lit, upper, lower, regexp_replace, to_date, monotonically_increasing_id, split, concat, arrays_zip, array

from pyspark.sql.types import IntegerType, StringType, FloatType, BooleanType

# Lettura dei dati

In [ ]:
files = [
  'DATASET_COMPLETO',
  'DATASET_PICCOLO',
  'CLASSE_A',
  'CLASSE_H'
]


dict_path =  {
  'DATASET' : "/FileStore/tables/DATASET_COMPLETO.csv",
  'CLASSE_A' : "/FileStore/tables/Classe_A.csv",
  'CLASSE_H' : "/FileStore/tables/Classe_H.csv"
}

## Dataset rielaborato

In [ ]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("multiLine", "true") \
  .option("engine", "python") \
  .option("encoding", "ISO-8859-1") \
  .load(dict_path['DATASET'])


display(df)

In [ ]:
print("Numero di entry DATASET_COMPLETO:", df.count())

## Dati Farmaci Classe A

In [ ]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

farmaci_gtA = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("multiLine", "true") \
  .option("engine", "python") \
  .option("encoding", "ISO-8859-1") \
  .load(dict_path['CLASSE_A'])

display(farmaci_gtA)

## Dati Farmaci Classe H

In [ ]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

farmaci_gtH = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("multiLine", "true") \
  .option("engine", "python") \
  .option("encoding", "ISO-8859-1") \
  .load(dict_path['CLASSE_H'])

display(farmaci_gtH)

## Aggiunta ID vistita

In [ ]:
df = df.withColumn("ID_VISITA", monotonically_increasing_id())

# Preparazione Dati visita

In [ ]:
df_visita = df.select('ID_VISITA', 'CODPAZ', 'SESSO', 'ETA', 'PESO', 'ALTEZZA', 'DATA_EVENTO', 'ESAMINATORE', 'COD_MEDICO_FIRMANTE')
display(df_visita)

ID_VISITA,CODPAZ,SESSO,ETA,PESO,ALTEZZA,DATA_EVENTO,ESAMINATORE,COD_MEDICO_FIRMANTE
0,7,null,null,80.0,163.0,2008-03-27,null,null
1,7,null,null,80.0,163.0,2008-06-10,null,null
2,1,null,null,null,null,2019-03-29,null,null
3,1,null,null,null,null,2019-05-03,null,null
4,1,null,null,null,165.0,2019-11-25,null,null
5,1,null,null,null,165.0,2020-07-15,null,1_61
6,1,null,null,null,165.0,2020-10-12,null,null
7,7,null,null,76.0,163.0,1998-11-11,null,null
8,7,null,null,76.0,163.0,2003-12-04,null,null
9,7,null,null,75.0,163.0,2005-09-02,null,null


In [ ]:
# Controllo valori nulli
temp = df_visita.select([count(when(col(c).isNull(), c)).alias(c) for c in df_visita.columns])

display(temp)

ID_VISITA,CODPAZ,SESSO,ETA,PESO,ALTEZZA,DATA_EVENTO,ESAMINATORE,COD_MEDICO_FIRMANTE
0,0,113345,113494,29402,11591,0,89975,66837


In [ ]:
# %fs rm -r "dbfs:/FileStore/preprocessing/DF_VISITA_COMPLETO"

res2: Boolean = true

In [ ]:
df_visita.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/preprocessing/DF_VISITA_COMPLETO")

# Preparazione Anamnesi e Diagnosi

In [ ]:
df_anam_dia = df.select("ID_VISITA", "ANAMNESI", "DIAGNOSI")
display(df_anam_dia)

ID_VISITA,ANAMNESI,DIAGNOSI
0,null,Ipertensione Arteriosa Grado I STADIO II. Ipercolesterolemia. Intolleranza glicidica Controllo farmacologico efficace. Pregressa Fibrillazione atriale.Gastropatia polipoide.
1,null,Ipertensione Arteriosa Grado I STADIO II. Ipercolesterolemia. Intolleranza glicidica Controllo farmacologico efficace. Pregressa Fibrillazione atriale.Gastropatia polipoide.
2,null,"Prossimo controllo prenotabile tra circa 12 mesi: Tel 0817462247 (da martedÃ¬ a giovedÃ¬ 08.30-13.00); mail: Cardiologia riabilitativa.FEDII@gmail.com CODICI PRESTAZIONI: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001) Torni al prossimo controllo con impegnativa del medico curante per Ã§Visita di controllo cardiologica per scompenso (cod. 89010.125), Elettrocardiogramma (cod. 89520.001), EcocolorDoppler cardiaco a riposo ( cod. 88723.001)Ã§."
3,null,CODICI IMPEGNATIVA PROSSIMO CONTROLLO: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001)
4,null,VISITA DI CONTROLLO CARDIOLOGICA (89010.014) ELETTROCARDIOGRAMMA (89520.001) Prossimo controllo prenotabile tra circa 12 mesi: Tel 0817462247 (da martedÃ¬ a giovedÃ¬ 10.00-12.30); mail: centroipertensionefedericoII@gmail.com
5,null,Prossimo controllo prenotabile tra circa 12 mesi: Tel 0817462247 (da martedÃ¬ a giovedÃ¬ 08.30-13.00); mail: centroipertensione.federicoII@gmail.com CODICI PRESTAZIONI: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001)
6,null,CODICI PRESTAZIONI: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001) Prossimo controllo prenotabile tra circa 12 mesi: inviare l`impegnativa alla mail: centroipertensionefedericoII@gmail.com
7,null,null
8,null,null
9,null,Ipertensione Arteriosa Grado I STADIO II Ipercolesterolemia. Intolleranza glicidica Controllo farmacologico efficace www.centroipertensione.it


In [ ]:
#%fs rm -r dbfs:/FileStore/preprocessing/DF_ANAM_DIA_COMPLETO

In [ ]:
df_anam_dia.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/preprocessing/DF_ANAM_DIA_COMPLETO")

# Preparazione farmaci

### Cleaning dei nomi

In [ ]:
for i in range(1,11):
  colonna = 'FARMACO' + str(i)
  colonna_new = 'FARMACO' + str(i) + '_CLEAN'
  df = df.withColumn(colonna_new, upper(col(colonna))).withColumn(colonna_new, regexp_replace(colonna_new, ' MG|\d+MG| MCG|\d+MCG| MGR|\d+MGR|\d+GR| GR| CPR| COMPRESSE| CEROTTO| CEROTTI| GOCCE| GEL| GTT| BUST| FL| UI| SPRAY|\d+CPR| CP|\d+CP|[^a-zA-Z0-9_ ]|\d+', ''))
  df = df.withColumn(colonna_new, regexp_replace(colonna_new, '^[ \t]+|[ \t]+$', ''))
  df = df.withColumn(colonna_new, regexp_replace(colonna_new, '[ \t]+', ' '))

In [ ]:
display(df)

CODPAZ,SESSO,ETA,PESO,ALTEZZA,DATA_EVENTO,COD_MEDICO_FIRMANTE,DIAGNOSI,FARMACO1,FARMACO2,FARMACO3,FARMACO4,FARMACO5,FARMACO6,FARMACO7,FARMACO8,FARMACO9,FARMACO10,DOSE_FARMACO1,DOSE_FARMACO2,DOSE_FARMACO3,DOSE_FARMACO4,DOSE_FARMACO5,DOSE_FARMACO6,DOSE_FARMACO7,DOSE_FARMACO8,DOSE_FARMACO9,DOSE_FARMACO10,ANAMNESI,ESAMINATORE,ID_VISITA,FARMACO1_CLEAN,FARMACO2_CLEAN,FARMACO3_CLEAN,FARMACO4_CLEAN,FARMACO5_CLEAN,FARMACO6_CLEAN,FARMACO7_CLEAN,FARMACO8_CLEAN,FARMACO9_CLEAN,FARMACO10_CLEAN
7,null,null,80.0,163.0,2008-03-27,null,Ipertensione Arteriosa Grado I STADIO II. Ipercolesterolemia. Intolleranza glicidica Controllo farmacologico efficace. Pregressa Fibrillazione atriale.Gastropatia polipoide.,RATACAND 16 mg,LOPRESOR,ARMOLIPID,MAALOX 400mg,null,null,null,null,null,null,1/2 al mattino,1/2 al mattino e alla sera,un/una alla sera,ogni 8 ore,null,null,null,null,null,null,null,null,0,RATACAND,LOPRESOR,ARMOLIPID,MAALOX,null,null,null,null,null,null
7,null,null,80.0,163.0,2008-06-10,null,Ipertensione Arteriosa Grado I STADIO II. Ipercolesterolemia. Intolleranza glicidica Controllo farmacologico efficace. Pregressa Fibrillazione atriale.Gastropatia polipoide.,RATACAND 16 mg,LOPRESOR,LIMPIDEX 15 mg,MAALOX PLUS,null,null,null,null,null,null,1/2 al mattino,1/2 al mattino e alla sera,ore 07.00 ore 22.00,ore 10.00 e ore 17.00,null,null,null,null,null,null,null,null,1,RATACAND,LOPRESOR,LIMPIDEX,MAALOX PLUS,null,null,null,null,null,null
1,null,null,null,null,2019-03-29,null,"Prossimo controllo prenotabile tra circa 12 mesi: Tel 0817462247 (da martedÃ¬ a giovedÃ¬ 08.30-13.00); mail: Cardiologia riabilitativa.FEDII@gmail.com CODICI PRESTAZIONI: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001) Torni al prossimo controllo con impegnativa del medico curante per Ã§Visita di controllo cardiologica per scompenso (cod. 89010.125), Elettrocardiogramma (cod. 89520.001), EcocolorDoppler cardiaco a riposo ( cod. 88723.001)Ã§.",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,null,null,null,null,null,null,null,null,null,null
1,null,null,null,null,2019-05-03,null,CODICI IMPEGNATIVA PROSSIMO CONTROLLO: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001),null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3,null,null,null,null,null,null,null,null,null,null
1,null,null,null,165.0,2019-11-25,null,VISITA DI CONTROLLO CARDIOLOGICA (89010.014) ELETTROCARDIOGRAMMA (89520.001) Prossimo controllo prenotabile tra circa 12 mesi: Tel 0817462247 (da martedÃ¬ a giovedÃ¬ 10.00-12.30); mail: centroipertensionefedericoII@gmail.com,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4,null,null,null,null,null,null,null,null,null,null
1,null,null,null,165.0,2020-07-15,1_61,Prossimo controllo prenotabile tra circa 12 mesi: Tel 0817462247 (da martedÃ¬ a giovedÃ¬ 08.30-13.00); mail: centroipertensione.federicoII@gmail.com CODICI PRESTAZIONI: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001),null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5,null,null,null,null,null,null,null,null,null,null
1,null,null,null,165.0,2020-10-12,null,CODICI PRESTAZIONI: VISITA DI CONTROLLO PER IPERTENSIONE (89010.064) ELETTROCARDIOGRAMMA (89520.001) Prossimo controllo prenotabile tra circa 12 mesi: inviare l`impegnativa alla mail: centroipertensionefedericoII@gmail.com,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null
7,null,null,76.0,163.0,1998-11-11,null,null,lopresor,null,null,null,null,null,null,null,null,null,un/una al mattino,null,null,null,null,null,null,null,null,null,null,null,7,LOPRESOR,null,null,null,null,null,null,null,null,null
7,null,null,76.0,163.0,2003-12

### Creazione dataframe con tutti i farmaci

In [ ]:
union = df.withColumn("farmaci", array('FARMACO1_CLEAN', 'FARMACO2_CLEAN', 'FARMACO3_CLEAN', 'FARMACO4_CLEAN', 'FARMACO5_CLEAN', 'FARMACO6_CLEAN', 'FARMACO7_CLEAN', 'FARMACO8_CLEAN', 'FARMACO9_CLEAN', 'FARMACO10_CLEAN')).select("farmaci").withColumn("farmaci", explode("farmaci"))

union = union.where(~union.farmaci.isNull()).where(union.farmaci != "").distinct()
display(union)

farmaci
PRAVASTATINA
RATACAND PLUS
IRRODAN
PERDIPINA
FARALZIN
ZOPRAZIDE
NIFEDICOR R
MEXAFORM
NITRODERM
OMINIC


In [ ]:
print("Numero di farmaci distinti nel dataset:", union.count())

Numero di farmaci distinti nel dataset: 4888

### Distanza di Levenshtein

In [ ]:
@udf(returnType= StringType()) 
def levenshtein(s, t):
    # Inizializza una matrica di zeri
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Popola la matrice iniziale con gli indici di ogni carattere di entrambe le stringhe
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Itera sulla matrice per calcolare il costo delle cancellazioni, inserimenti e/o sostituzioni 
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                # Se i caratteri sono gli stessi nelle due stringhe in una data posizione [i,j] allora il costo è 0
                cost = 0 
            else:
                cost = 2
            distance[row][col] = np.min([distance[row-1][col] + 1,  # Costo della cancellazione
                                 distance[row][col-1] + 1,          # Costo dell'inserimento
                                 distance[row-1][col-1] + cost])    # Costo della sostituzione

    # Calcolo del Levenshtein Distance Ratio
    Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
    return str(Ratio)

#### Creazione ground truth

In [ ]:
split_col = split(farmaci_gtA['Denominazione e Confezione'], '\\*')
farmaci_gtA = farmaci_gtA.withColumn('Farmaco', split_col.getItem(0)).select('Farmaco').distinct()
split_col = split(farmaci_gtH['Denominazione e Confezione'], '\\*')
farmaci_gtH = farmaci_gtH.withColumn('Farmaco', split_col.getItem(0)).select('Farmaco').distinct()
farmaci_gt = farmaci_gtA.select('Farmaco').union(farmaci_gtH.select('Farmaco')).distinct().withColumnRenamed('Farmaco', 'farmaci_corretti')
print("Numero di farmaci ground truth:", farmaci_gt.count())

Numero di farmaci ground truth: 3210

In [ ]:
gt = union.join(farmaci_gt.withColumnRenamed('farmaci_corretti', 'farmaci'), on='farmaci', how='inner')
display(gt)
print("Numero di farmaci ground truth:", gt.count())

farmaci
PRAVASTATINA
RATACAND PLUS
FARALZIN
SULIDAMOR
KROVANEG
ZOPRAZIDE
INTRON A
ADROVANCE
AMINOMAL
SOPAVI


Numero di farmaci ground truth: 1011

In [ ]:
anti = union.join(gt, on="farmaci", how='anti')
print("Numero di farmaci del dataset non presenti nella GT:", anti.count())

Numero di farmaci del dataset non presenti nella GT: 3877

#### Calcolo tabella di similarità

In [ ]:
cross = anti.select("farmaci").withColumnRenamed('farmaci', 'farmaci_misspelled').crossJoin(gt.withColumnRenamed("farmaci", "farmaci_corretti").select('farmaci_corretti'))
similarity = cross.withColumn("result", levenshtein(cross.farmaci_misspelled, cross.farmaci_corretti))
similarity = similarity.withColumn("result", similarity["result"].cast("float"))
similarity_09 = similarity.filter(similarity.result >= 0.9)

In [ ]:
# %fs rm -r "/FileStore/preprocessing/similarity_09-gt-efficiente-dataset-completo"

In [ ]:
similarity_09.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("/FileStore/preprocessing/similarity_09-gt-efficiente-dataset-completo")

### Correzione nomi farmaci

In [ ]:
file_location = "/FileStore/tables/similarity_09_gt_efficiente_dataset_completo.csv"
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

similarity_09 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(similarity_09)

farmaci_misspelled,farmaci_corretti,result
PRAVSTATINA,PRAVASTATINA,0.95652175
RATACANDPLUS,RATACAND PLUS,0.96
RATCAND PLUS,RATACAND PLUS,0.96
RATACND PLUS,RATACAND PLUS,0.96
INTRONA,INTRON A,0.93333334
EFFERALGAN,COEFFERALGAN,0.90909094
PRIMOLUTNOR,PRIMOLUT NOR,0.95652175
TRADIUR,TORADIUR,0.93333334
TOADIUR,TORADIUR,0.93333334
COVERTEN,CONVERTEN,0.9411765


In [ ]:
print("Numero di corrispondenze con confidenza >0.9:", similarity_09.count())

Numero di corrispondenze con confidenza >0.9: 581

#### Creazione tabella delle corrispondenze

In [ ]:
temp = similarity_09.groupBy("farmaci_misspelled").max("result")
mapping = temp.join(similarity_09.withColumnRenamed("result", "max(result)"), on=["max(result)", "farmaci_misspelled"]).orderBy("farmaci_misspelled")
display(mapping)

max(result),farmaci_misspelled,farmaci_corretti
0.962963,AATORVASTATINA,ATORVASTATINA
0.9411765,ABSORCORL,ABSORCOL
0.9411765,ACCURETC,ACCURETIC
0.93333334,ACEQUID,ACEQUIDE
0.9411765,ACESITEM,ACESISTEM
0.96,ACIFDO FOLICO,ACIDO FOLICO
0.9411765,ACRIPTIN,ASCRIPTIN
0.93333334,ACUPRIN,ACCUPRIN
0.9411765,ACURETIC,ACCURETIC
0.90909094,ADALA CRON,ADALAT CRONO


In [ ]:
print("Numero di mapping decretati:", mapping.count())

Numero di mapping decretati: 580

#### Sostituzione misspelling

In [ ]:
df_farmaci = df.drop("SESSO", "ETA", "PESO", "ALTEZZA", "ANAMNESI", "DIAGNOSI", "CODPAZ", "COD_MEDICO_FIRMANTE", "DATA_EVENTO")

df_zip = df_farmaci.withColumn("FARMACO", array('FARMACO1_CLEAN', 'FARMACO2_CLEAN', 'FARMACO3_CLEAN', 'FARMACO4_CLEAN', 'FARMACO5_CLEAN', 'FARMACO6_CLEAN', 'FARMACO7_CLEAN', 'FARMACO8_CLEAN', 'FARMACO9_CLEAN', 'FARMACO10_CLEAN')).withColumn("CONFEZIONE", array('FARMACO1', 'FARMACO2', 'FARMACO3', 'FARMACO4', 'FARMACO5', 'FARMACO6', 'FARMACO7', 'FARMACO8', 'FARMACO9', 'FARMACO10')).withColumn("DOSE", array('DOSE_FARMACO1', 'DOSE_FARMACO2', 'DOSE_FARMACO3', 'DOSE_FARMACO4', 'DOSE_FARMACO5', 'DOSE_FARMACO6', 'DOSE_FARMACO7', 'DOSE_FARMACO8', 'DOSE_FARMACO9', 'DOSE_FARMACO10')).select("ID_VISITA", "FARMACO", "CONFEZIONE", "DOSE")

df_farmaci = df_zip.withColumn("tmp", arrays_zip("FARMACO", "CONFEZIONE", "DOSE")) \
  .withColumn("tmp", explode("tmp")) \
  .select("ID_VISITA", "tmp.FARMACO", "tmp.CONFEZIONE", "tmp.DOSE") \
  .dropna(subset="FARMACO")

In [ ]:
display(df_farmaci)

ID_VISITA,FARMACO,CONFEZIONE,DOSE
0,RATACAND,RATACAND 16 mg,1/2 al mattino
0,LOPRESOR,LOPRESOR,1/2 al mattino e alla sera
0,ARMOLIPID,ARMOLIPID,un/una alla sera
0,MAALOX,MAALOX 400mg,ogni 8 ore
1,RATACAND,RATACAND 16 mg,1/2 al mattino
1,LOPRESOR,LOPRESOR,1/2 al mattino e alla sera
1,LIMPIDEX,LIMPIDEX 15 mg,ore 07.00 ore 22.00
1,MAALOX PLUS,MAALOX PLUS,ore 10.00 e ore 17.00
7,LOPRESOR,lopresor,un/una al mattino
8,LOPRESOR,LOPRESOR,null


In [ ]:
temp = df_farmaci.join(mapping.withColumnRenamed("farmaci_misspelled", "FARMACO"), on="FARMACO", how="left_outer")
finale = temp.withColumn("FARMACO", when(temp.farmaci_corretti.isNull(), temp["FARMACO"]).otherwise(temp["farmaci_corretti"])).withColumnRenamed("FARMACO", "FARMACO_FINALE")

df_farmaci = finale.drop("max(result)", "farmaci_corretti").drop_duplicates()

In [ ]:
print("Numero di farmaci misspelled corretti:", temp.where(~temp.farmaci_corretti.isNull()).count())
print("Numero di farmaci misspelled distinct corretti:", temp.where(~temp.farmaci_corretti.isNull()).select("Farmaco").distinct().count())

Numero di farmaci misspelled corretti: 12731
Numero di farmaci misspelled distinct corretti: 580

In [ ]:
display(df_farmaci)

FARMACO_FINALE,ID_VISITA,CONFEZIONE,DOSE
SIRDALUD,34,SIRDALUD,un/una al mattino e sera
NORVASC,143,NORVASC 5,1 cp al pomeriggio
ENAPREN,207,ENAPREN 20,un/una al mattino
TOTALIP,242,TOTALIP 20,un/una alla sera
ANSIMAR SCIROPPO,81,Ansimar sciroppo,un misurino la sera
PLAVIX,238,PLAVIX 75,1 dopo il pasto
PRITORPLUS,268,PRITOR 80/12.5 PLUS,un/una al mattino
METFORAL,287,Metforal 1000,1/2 cp a pranzo e cena
ISOPTIN,295,Isoptin 180,Ogni 12 ore
METFORAL,298,Metforal 1000,1/2 cp a pranzo e cena


In [ ]:
#%fs rm -r dbfs:/FileStore/preprocessing/DF_FARMACI_COMPLETO

In [ ]:
df_farmaci.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/preprocessing/DF_FARMACI_COMPLETO")